In [1]:
import pandas as pd
import os
import re
import spacy
import en_core_web_sm
from spacy.matcher import Matcher
from IPython.display import clear_output


/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)
/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)


In [2]:
data_path = '/Users/kylieleblancKylie/domino/dofis/data/'
data = pd.read_csv(os.path.join(data_path, 'plans', 'doi_final_wtext.csv'),
                  sep=",")
data = data[['district', 'link', 'text', 'reg25_0811', 'reg25_081', 'reg25_0812', 'reg25_082',
            'reg25_112', 'reg25_113', 'reg25_111', 
            'reg21_003', 'reg21_053', 'reg21_057',
            'reg21_102', 'reg21_401', 'reg21_352', 'reg21_354',
            'reg25_092', 'reg37_0012', 'reg25_036']]

In [3]:
similar_laws = {'25.0811': ['25.0811', '25_081', '25.0812', '25.082'],
               '25.112': ['25.112', '25.113', '25.111'], 
               '21.003': ['21.003', '21.053', '21.044', '21.057', '21.055'],
               '21.102': ['21.102','21.401', '21.352'],
                '25.092': ['25.092']}
law_name = {'reg25_0812': '25.0812', 
           'reg21_003': '21.003'}

In [4]:
stubnames = sorted(
    set([match[0] for match in data.columns.str.findall(
    r'reg.*').values if match != [] ])
    )

In [5]:
long = pd.melt(data, id_vars=['district', 'link', 'text'], value_vars=stubnames)
long = long[long.value == 1]
print('length= ', len(long))
long.head()

length=  4485


,district,link,text,variable,value
1,Abernathy ISD,https://1.cdn.edl.io/epGD4mZXjWUcPl8yA7IhlcDol...,Abernathy ISD District of Innovation Plan – Dr...,reg21_003,1
2,Abilene ISD,https://www.abileneisd.org/wp-content/uploads/...,0 ABILENE INDEPENDENT SCHOOL DISTRICT Local In...,reg21_003,1
3,Academy ISD,https://4.files.edl.io/1a8f/06/29/18/204245-44...,District of Innovation Plan Developed in coope...,reg21_003,1
4,Adrian ISD,http://www.adrianisd.net/UserFiles/Servers/Ser...,Adrian ISD District of Innovation Plan House B...,reg21_003,1
5,Agua Dulce ISD,https://tx02206063.schoolwires.net/cms/lib/TX0...,Microsoft Word - DOI 2017-2022.docx Agua Dulce...,reg21_003,1


In [6]:
nlp = spacy.load('en_core_web_sm')
law_shapes = [i*'d' + '.' + j*'d' + k*'x' for i in range(1, 4) for j in range(3,5) for k in range(3)]
law_shape_patterns = [[{'SHAPE':shape}, {'ORTH':'%', 'OP':'!'}] for shape in law_shapes] # could add {'SHAPE':'§', 'OP':'*'},

matcher = Matcher(nlp.vocab)
matcher.add("ExplicitLaw", None, *law_shape_patterns)

/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)


In [7]:
def get_phrase(text, regulation):
    phrase = ''
    # Get location of regulation matches
    doc = nlp(text)
    matches = matcher(doc)
    locs = []
    for match in matches:
        locs.append(match[1])
    locs.append(len(doc) - 1)
    # Get phrases between one match and the next
    i = 0
    for loc in locs[0:-1]: 
        token = doc[loc].text
        if token.startswith(regulation):
            start = locs[i]
            j = i + 1
            end = locs[j]
            while doc[end].text in similar_laws[regulation]:
                j = j + 1
                end = locs[j]
            phrase = phrase + "|" + str(doc[start:end])
        i = i + 1
    return phrase
get_phrase("21.003 45.211 21.003 21.053, 21.044 This is about teacher certification okay", regulation = '21.003')

'|21.003|21.003 21.053, 21.044 This is about teacher certification'

# Certification

In [8]:
certification = long[long.variable == 'reg21_003']
certification.head()

,district,link,text,variable,value
1,Abernathy ISD,https://1.cdn.edl.io/epGD4mZXjWUcPl8yA7IhlcDol...,Abernathy ISD District of Innovation Plan – Dr...,reg21_003,1
2,Abilene ISD,https://www.abileneisd.org/wp-content/uploads/...,0 ABILENE INDEPENDENT SCHOOL DISTRICT Local In...,reg21_003,1
3,Academy ISD,https://4.files.edl.io/1a8f/06/29/18/204245-44...,District of Innovation Plan Developed in coope...,reg21_003,1
4,Adrian ISD,http://www.adrianisd.net/UserFiles/Servers/Ser...,Adrian ISD District of Innovation Plan House B...,reg21_003,1
5,Agua Dulce ISD,https://tx02206063.schoolwires.net/cms/lib/TX0...,Microsoft Word - DOI 2017-2022.docx Agua Dulce...,reg21_003,1


In [9]:
phrases = []
for text in certification.text:
    phrase = get_phrase(text, '21.003')
    phrases.append(phrase)

In [10]:
certification['phrase'] = phrases

/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [11]:
certification.to_csv(os.path.join(data_path, 'clean', 'certification_phrases.csv'),
                  sep=",")

# Can Start Here

In [4]:
docs = pd.read_csv(os.path.join(data_path, 'clean', 'certification_phrases.csv'),
                  sep=",")
docs.head()

,Unnamed: 0,district,link,text,variable,value,phrase
0,1,Abernathy ISD,https://1.cdn.edl.io/epGD4mZXjWUcPl8yA7IhlcDol...,Abernathy ISD District of Innovation Plan – Dr...,reg21_003,1,|21.003; TEC§ 21.057 The principal may submit ...
1,2,Abilene ISD,https://www.abileneisd.org/wp-content/uploads/...,0 ABILENE INDEPENDENT SCHOOL DISTRICT Local In...,reg21_003,1,|21.003 (a); T.E.C. Section §21.053 (a)-(b); C...
2,3,Academy ISD,https://4.files.edl.io/1a8f/06/29/18/204245-44...,District of Innovation Plan Developed in coope...,reg21_003,1,"|21.003, 21.053,|21.003, 21.053,"
3,4,Adrian ISD,http://www.adrianisd.net/UserFiles/Servers/Ser...,Adrian ISD District of Innovation Plan House B...,reg21_003,1,|21.003 DK LEGAL Current TEC §21.003 states a ...
4,5,Agua Dulce ISD,https://tx02206063.schoolwires.net/cms/lib/TX0...,Microsoft Word - DOI 2017-2022.docx Agua Dulce...,reg21_003,1,|21.003. CERTIFICATION REQUIRED. (a) A person ...


In [5]:
docs_to_label = docs.sample(n = 50, random_state = 45)
docs_to_label = docs_to_label[['phrase', 'link']]
docs_to_label = docs_to_label.rename({'phrase': 'text', 'link': 'source'}, axis = 1)
#docs_to_label.to_csv(os.path.join(data_path, 'plans', 'certification_to_label.csv'), header = True)

In [6]:
relabel_docs = True

In [7]:
len(docs_to_label)

50

In [26]:
if relabel_docs:
    shortage = []
    outofsubject = []
    electives = []
    for text, source in zip(docs_to_label.text[0:10], docs_to_label.source[0:10]):
        show = str(source) + str(text)
        
        annotation = input(show + "Mentions paperwork or hard-to-fill?")
        shortage.append(annotation)
        
        annotation = input("Mentions certified not in subject?")
        outofsubject.append(annotation)
        
        annotation = input("Mentions CTE, dual-credit, or elective courses?")
        electives.append(annotation)
                           
        clear_output()

In [31]:
electives = electives[0:10]
shortage = shortage[0:10]
outofsubject = outofsubject[0:10]

In [32]:
docs_to_label.text[10:20]

216    |21.003 - A person may not be employed as a te...
64     |21.003) Currently In the event a district can...
48     |21.003) Currently In the event a district can...
63     |21.003 Current Law: In the event a district c...
18                                                   NaN
446    |21.003, 21.053 and 21.057) Sec. 21.003. CERTI...
565    |21.003, 21.053, and 21.057) DK (LEGAL) State ...
61     |21.003) Current Statute: A person may not be ...
102                                                  NaN
356    |21.003 Certification Required 2. TEC|21.003 r...
Name: text, dtype: object

In [35]:
if relabel_docs:
        for text, source in zip(docs_to_label.text[10:20], docs_to_label.source[10:20]):
            show = str(source) + str(text)
            annotation = input(show + "Mentions paperwork or hard-to-fill?")
            shortage.append(annotation)

            annotation = input("Mentions certified not in subject?")
            outofsubject.append(annotation)

            annotation = input("Mentions CTE, dual-credit, or elective courses?")
            electives.append(annotation)

            clear_output()

In [36]:
if relabel_docs:
        for text, source in zip(docs_to_label.text[20:30], docs_to_label.source[20:30]):
            show = str(source) + str(text)
            annotation = input(show + "Mentions paperwork or hard-to-fill?")
            shortage.append(annotation)

            annotation = input("Mentions certified not in subject?")
            outofsubject.append(annotation)

            annotation = input("Mentions CTE, dual-credit, or elective courses?")
            electives.append(annotation)

            clear_output()

In [42]:
if relabel_docs:
        for text, source in zip(docs_to_label.text[30:40], docs_to_label.source[30:40]):
            show = str(source) + str(text)
            annotation = input(show + "Mentions paperwork or hard-to-fill?")
            shortage.append(annotation)

            annotation = input("Mentions certified not in subject?")
            outofsubject.append(annotation)

            annotation = input("Mentions CTE, dual-credit, or elective courses?")
            electives.append(annotation)

            clear_output()

In [43]:
if relabel_docs:
        for text, source in zip(docs_to_label.text[40:50], docs_to_label.source[40:50]):
            show = str(source) + str(text)
            annotation = input(show + "Mentions paperwork or hard-to-fill?")
            shortage.append(annotation)

            annotation = input("Mentions certified not in subject?")
            outofsubject.append(annotation)

            annotation = input("Mentions CTE, dual-credit, or elective courses?")
            electives.append(annotation)

            clear_output()

In [72]:
docs_to_label['shortage'] =  shortage
docs_to_label['outofsubject'] = outofsubject
docs_to_label['electives'] = electives

In [73]:
docs_to_label['shortage'] = pd.to_numeric(docs_to_label['shortage'], errors='coerce')
docs_to_label['outofsubject'] = pd.to_numeric(docs_to_label['outofsubject'], errors='coerce')
docs_to_label['electives'] = pd.to_numeric(docs_to_label['electives'], errors='coerce')

In [76]:
docs_to_label
print("Proportion mentioning emergency paperwork: ", docs_to_label['shortage'].mean())
print("Proportion mentioning CTE: ", docs_to_label['electives'].mean())
print("Proportion mentioning out of subject certification: ", docs_to_label['outofsubject'].mean())
docs_to_label.head()

Proportion mentioning emergency paperwork:  0.32653061224489793
Proportion mentioning CTE:  0.84
Proportion mentioning out of subject certification:  0.52


,text,source,shortage,outofsubject,electives
524,|21.003) Currently In the event a district can...,http://images.pcmac.org/Uploads/TrentonISD/Tre...,1.0,0,0
159,|21.003) (DK Exhibit) (DK Legal) (DK Local) TE...,http://www.emsisd.com/cms/lib/TX21000533/Centr...,0.0,0,1
133,"|21.003, §21.044, §21.053 and §21.057) State l...",https://s3.amazonaws.com/scschoolfiles/1170/cr...,0.0,0,1
535,"|21.003, §21.044, §21.053, §21.055) As a small...",https://d3jc3ahdjad7x7.cloudfront.net/F5HPSEGE...,1.0,0,1
212,|21.003 Current Law: In the event a district c...,https://1.cdn.edl.io/hccNlwvPnDQR6L8Rr8kSyY5uc...,0.0,1,1


In [55]:
docs_to_label

,text,source,shortage,outofsubject,electives
524,|21.003) Currently In the event a district can...,http://images.pcmac.org/Uploads/TrentonISD/Tre...,1,0,0
159,|21.003) (DK Exhibit) (DK Legal) (DK Local) TE...,http://www.emsisd.com/cms/lib/TX21000533/Centr...,0,0,1
133,"|21.003, §21.044, §21.053 and §21.057) State l...",https://s3.amazonaws.com/scschoolfiles/1170/cr...,0,0,1
535,"|21.003, §21.044, §21.053, §21.055) As a small...",https://d3jc3ahdjad7x7.cloudfront.net/F5HPSEGE...,1,0,1
212,|21.003 Current Law: In the event a district c...,https://1.cdn.edl.io/hccNlwvPnDQR6L8Rr8kSyY5uc...,0,1,1
448,"|21.003, §21.044, §21.053 and §21.057) State l...",https://docs.google.com/document/d/1H8XRcRNhFn...,1,0,1
210,|21.003) CERTIFICATION REQUIRED. (a) A person ...,http://www.godleyisd.net/apps/pages/index.jsp?...,1,0,1
571,|21.003 states that a person may not be employ...,http://www.wyliebulldogs.org/cms/One.aspx?port...,0,1,1
445,"|21.003, 21.053, 21.057, and 19 TAC Chapter 23...",http://www.rivieraisd.us/UserFiles/Servers/Ser...,1,1,1
460,"|21.003, 21.057) In the event a district canno...",http://www.sabinalstorage.tech/requiredposting...,1,1,1


In [25]:
cte

['0', '1', '1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']

In [ ]:
if relabel_docs:
    cte = []
    for text, source in zip(docs_to_label.text, docs_to_label.source):
        show = source + text
        annotation = input(show)
        paperwork.append(annotation)
        clear_output()

In [ ]:
if relabel_docs:
    flexibility = []
    for text, source in zip(docs_to_label.text, docs_to_label.source):
        show = source + text
        annotation = input(show)
        paperwork.append(annotation)
        clear_output()